In [2]:
import os
import numpy as np
from yael import ynumpy

In [3]:
train_desc_path = "/home/joan/workspace/MCV_M3/M3_project/SavePath/desc_fisher/train.dat"
test_desc_path = "/home/joan/workspace/MCV_M3/M3_project/SavePath/desc_fisher/test.dat"

In [4]:
import pickle

In [5]:
train_desc = pickle.load(open(train_desc_path, 'rb'))
test_desc = pickle.load(open(test_desc_path, 'rb'))
all_train_desc = np.vstack(train_desc)

In [26]:
k = 64
n_sample = k * 5000

# choose n_sample descriptors at random
sample_indices = np.random.choice(all_train_desc.shape[0], n_sample)
sample = all_train_desc[sample_indices]

# until now sample was in uint8. Convert to float32
sample = sample.astype('float32')

# until now sample was in uint8. Convert to float32
gmm = ynumpy.gmm_learn(sample, k)

(963072, 128)
(320000, 128)


In [27]:
train_fvs = []

for image_desc in train_desc:
    fv = ynumpy.fisher(gmm, image_desc, include = 'mu')
    train_fvs.append(fv)

In [50]:
test_fvs = []

for image_desc in test_desc:
    fv = ynumpy.fisher(gmm, image_desc, include = 'mu')
    test_fvs.append(fv)

In [51]:
train_data_path = "/home/joan/workspace/MCV_M3/M3_project/SavePath/data_fisher/train.dat"
test_data_path = "/home/joan/workspace/MCV_M3/M3_project/SavePath/data_fisher/test.dat"

# Save computed data
pickle.dump(train_fvs, open(train_data_path, 'wb'), protocol=2)
pickle.dump(test_fvs, open(test_data_path, 'wb'), protocol=2)

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import normalize

In [ ]:
import os
from glob import glob
from random import shuffle

from sklearn.preprocessing import LabelEncoder


class DatasetManager():

    def __init__(self, path):
        self.path = path
        self.label_encoder = LabelEncoder()

    def load_dataset(self,fraction=1):
        self.train_img_paths = glob(os.path.join(self.path, "train", "*", "*.jpg"))
        self.test_img_paths = glob(os.path.join(self.path, "test", "*", "*.jpg"))
        self.train_labels = [os.path.split(os.path.split(img_path)[0])[-1] for img_path in self.train_img_paths]
        self.test_labels = [os.path.split(os.path.split(img_path)[0])[-1] for img_path in self.test_img_paths]

        self.transform_labels()

        return self.train_img_paths, self.train_labels, self.test_img_paths, self.test_labels

    def transform_labels(self):
        self.label_encoder.fit(self.train_labels)
        self.train_labels = self.label_encoder.transform(self.train_labels)
        self.test_labels = self.label_encoder.transform(self.test_labels)

    def inverse_transform_labels(self):
        self.train_labels = self.label_encoder.inverse_transform(self.train_labels)
        self.test_labels = self.label_encoder.inverse_transform(self.test_labels)


In [ ]:
train_data_path = "/home/joan/workspace/MCV_M3/M3_project/SavePath/data_fisher/train.dat"
test_data_path = "/home/joan/workspace/MCV_M3/M3_project/SavePath/data_fisher/test.dat"
dataset = DatasetManager("/home/joan/workspace/MCV_M3/M3_project/Databases/MIT_split")
_, train_labels, _, test_labels = dataset.load_dataset()
train_fvs = pickle.load(open(train_data_path, 'rb'))
test_fvs = pickle.load(open(test_data_path, 'rb'))
train_data = np.array(train_fvs)
test_data = np.array(test_fvs)

In [46]:
K_FOLDS = 5
PARAMETERS = {
    'C': [1e-1],
    'gamma': ["scale"],
    'kernel': ["rbf"]}

cv = GridSearchCV(SVC(), param_grid=PARAMETERS, cv=K_FOLDS, n_jobs=-1, verbose=5)
cv.fit(train_data, train_labels)

(1881, 8192)
1881
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.2min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'kernel': ['rbf'], 'C': [0.1], 'gamma': ['scale']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [54]:
train_score = cv.score(train_data, train_labels)
test_score = cv.score(test_data, test_labels)
        
print("Train accuracy score: {}\nTest accuracy score: {}\nBest params: {}\n".format(train_score, test_score, cv.best_params_))
print("All results: {}".format(cv.cv_results_))

Train accuracy score: 0.548112706007
Test accuracy score: 0.484510532838
Best params: {'kernel': 'rbf', 'C': 0.1, 'gamma': 'scale'}

All results: {'std_train_score': array([0.00615506]), 'split4_test_score': array([0.39247312]), 'split0_train_score': array([0.45339547]), 'rank_test_score': array([1], dtype=int32), 'param_gamma': masked_array(data=['scale'],
             mask=[False],
       fill_value='?',
            dtype=object), 'split2_train_score': array([0.45049834]), 'std_score_time': array([0.11076805]), 'split4_train_score': array([0.45593108]), 'split2_test_score': array([0.38829787]), 'mean_score_time': array([11.16088276]), 'mean_fit_time': array([73.16375642]), 'param_kernel': masked_array(data=['rbf'],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_C': masked_array(data=[0.1],
             mask=[False],
       fill_value='?',
            dtype=object), 'std_test_score': array([0.00563429]), 'split1_train_score': array([0.43808256]), '